In [1]:
import pandas as pd
import os
import numpy as np
import openpyxl
import psycopg2

In [2]:
df= pd.read_excel("Contacts.xlsx")
df.head()

c:\Users\Jetstreamafrica\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Created Time,Last Modified Time,Display Name,Company Name,Salutation,First Name,Last Name,Phone,Currency Code,Notes,...,EmailID,MobilePhone,Contact ID,Contact Name,Contact Type,Taxable,Tax Name,Tax Percentage,Contact Address ID,Source
0,2021-07-08 20:04:41,2022-01-20 05:16:24,VLEC Delivery Services - GHS,VLEC Delivery Services - GHS,NaN,Bonsu,NaN,NaN,GHS,NaN,...,infor@victoriouscargo.nl,+233204679781,2620362000000078039,VLEC Delivery Services - GHS,customer,True,NaN,NaN,2620362000000078039,user
1,2021-07-08 20:55:37,2022-02-07 22:11:00,PharmaConnect Nigeria - NGN,PharmaConnect Nigeria - NGN,Mr.,Malachi,Sobowale,NaN,NGN,NaN,...,malachi@mpharma.com,2348170339709,2620362000000078149,PharmaConnect Nigeria - NGN,customer,True,NaN,NaN,2620362000000078149,user
2,2021-07-08 21:09:34,2022-01-20 05:16:06,Spill Shea Butter - USD,Spill Shea Butter - USD,Mr,Hamza,Mogtari Mabruka,NaN,USD,NaN,...,hamzamaby11@gmail.com,0502845460,2620362000000078195,Spill Shea Butter - USD,customer,True,NaN,NaN,2620362000000078195,user
3,2021-07-08 21:34:43,2022-01-20 05:16:39,Organic Pavillion - USD,Organic Pavillion - USD,Mr.,Jerry,Adams,NaN,USD,NaN,...,NaN,0553763787,2620362000000082027,Organic Pavillion - USD,customer,True,NaN,NaN,2620362000000082027,user
4,2021-07-08 23:14:46,2021-10-25 03:16:54,Hans & Ezra - USD,Hans & Ezra - USD,Mr.,David,Osei,NaN,USD,NaN,...,david.ok@hansezra.com,NaN,2620362000000082412,Hans & Ezra - USD,customer,True,NaN,NaN,2620362000000082412,user


In [3]:
df.columns = [x.lower().replace(" ","_").replace(".","")for x in df.columns]
df.columns

Index(['created_time', 'last_modified_time', 'display_name', 'company_name',
       'salutation', 'first_name', 'last_name', 'phone', 'currency_code',
       'notes', 'website', 'status', 'opening_balance', 'exchange_rate',
       'branch_id', 'branch_name', 'credit_limit', 'customer_sub_type',
       'billing_attention', 'billing_address', 'billing_street2',
       'billing_city', 'billing_state', 'billing_country', 'billing_code',
       'billing_phone', 'billing_fax', 'shipping_attention',
       'shipping_address', 'shipping_street2', 'shipping_city',
       'shipping_state', 'shipping_country', 'shipping_code', 'shipping_phone',
       'shipping_fax', 'skype_identity', 'facebook', 'twitter', 'department',
       'designation', 'price_list', 'payment_terms', 'payment_terms_label',
       'tax_type', 'last_sync_time', 'owner_name', 'primary_contact_id',
       'emailid', 'mobilephone', 'contact_id', 'contact_name', 'contact_type',
       'taxable', 'tax_name', 'tax_percentage', 'con

In [4]:
df.dtypes

created_time            object
last_modified_time      object
display_name            object
company_name            object
salutation              object
first_name              object
last_name               object
phone                   object
currency_code           object
notes                  float64
website                 object
status                  object
opening_balance        float64
exchange_rate          float64
branch_id              float64
branch_name            float64
credit_limit           float64
customer_sub_type       object
billing_attention       object
billing_address         object
billing_street2         object
billing_city            object
billing_state           object
billing_country         object
billing_code           float64
billing_phone           object
billing_fax            float64
shipping_attention      object
shipping_address        object
shipping_street2       float64
shipping_city           object
shipping_state          object
shipping

In [6]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'varchar',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'varcahr',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [7]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'created_time varchar, last_modified_time varchar, display_name varchar, company_name varchar, salutation varchar, first_name varchar, last_name varchar, phone varchar, currency_code varchar, notes float, website varchar, status varchar, opening_balance float, exchange_rate float, branch_id float, branch_name float, credit_limit float, customer_sub_type varchar, billing_attention varchar, billing_address varchar, billing_street2 varchar, billing_city varchar, billing_state varchar, billing_country varchar, billing_code float, billing_phone varchar, billing_fax float, shipping_attention varchar, shipping_address varchar, shipping_street2 float, shipping_city varchar, shipping_state varchar, shipping_country varchar, shipping_code float, shipping_phone float, shipping_fax float, skype_identity float, facebook float, twitter float, department float, designation float, price_list float, payment_terms varcahr, payment_terms_label varchar, tax_type varchar, last_sync_time float, owner_name f

In [24]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [25]:
cursor.execute("drop table if exists contacts;")

In [26]:
cursor.execute("create table contacts \
    (created_time varchar, last_modified_time varchar, display_name varchar, company_name varchar, salutation varchar, first_name varchar, last_name varchar, phone varchar, currency_code varchar, notes varchar, website varchar, status varchar, opening_balance float, exchange_rate float, branch_id varchar, branch_name varchar, credit_limit float, customer_sub_type varchar, billing_attention varchar, billing_address varchar, billing_street2 varchar, billing_city varchar, billing_state varchar, billing_country varchar, billing_code float, billing_phone varchar, billing_fax float, shipping_attention varchar, shipping_address varchar, shipping_street2 float, shipping_city varchar, shipping_state varchar, shipping_country varchar, shipping_code float, shipping_phone float, shipping_fax float, skype_identity varchar, facebook varchar, twitter varchar, department varchar, designation varchar, price_list float, payment_terms varchar, payment_terms_label varchar, tax_type varchar, last_sync_time varchar, owner_name varchar, primary_contact_id float, emailid varchar, mobilephone varchar, contact_id varchar, contact_name varchar, contact_type varchar, taxable varchar, tax_name varchar, tax_percentage float, contact_address_id varchar, source varchar)")

In [27]:
df.to_csv('contacts', header=df.columns, index=False, encoding='utf-8')
my_file = open("contacts")
print('file opened in memory')

file opened in memory


In [28]:
SQL_STATEMENT = """
COPY contacts FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [29]:
cursor.execute("grant select on table contacts to public")
conn.commit()

cursor.close()
print('table contacts imported successfully')

table contacts imported successfully
